# Two-Sample t-Test

These slides might help understanding the concepts: https://docs.google.com/presentation/d/1-YZXJ2D6ghztOpXw3DxA1lc9MYxKo-VVsa_wclZQFGY/edit?usp=sharing 

Watch this short before starting the code along, as it introduces the example we will follow: https://www.youtube.com/watch?v=NkGvw18zlGQ

In [ ]:
import scipy.stats as stats
import numpy as np

We've generated 2 samples for tomato plant highs with the same means and standard deviations of fields A and B from Khan Academy's video*.


*You only need the mean and the std to perform a statistical test following the equations in the "formal approach", but it's nicer to have the full data if you want to see how the tests are performed using simulations (i.e. the "resampling approach") or using Scipy's functions. Please, do not worry about how the data was generated: it does not matter at all for learning statistics! The exact lengths of the plants could have taken other values, but as long as each sample has the same mean and standard deviation, the results of the statistical tests are going to be the same.

In [ ]:
f_a = np.array([1.53021758, 0.70571215, 2.01610148, 1.14367405, 0.93970563,
       1.74358147, 1.72979421, 0.99173825, 1.30784819, 0.18865752,
       1.92501786, 1.79597301, 1.77666206, 0.54937259, 1.13296132,
       1.40105918, 1.60272671, 1.44154697, 1.9605791 , 0.52757822,
       1.19867684, 0.99201533])


print("mean field A:", np.mean(f_a))
print("standard deviation field A:", np.std(f_a))

In [ ]:
f_b = np.array([1.65802641, 1.76603167, 1.89544547, 1.45920841, 1.80266623,
       0.80989183, 1.54506744, 1.85769076, 1.48064793, 1.7310313 ,
       1.91207357, 1.71378148, 1.70911519, 1.56339253, 1.63741389,
       1.50316156, 1.85250241, 2.15778815, 1.62285988, 1.43006622,
       1.61084258, 0.97720672, 1.67433766, 1.03085296])


print("mean field B:", np.mean(f_b))
print("standard deviation field B:", np.std(f_b))

In [ ]:
# difference between the 2 fields:
difference = abs(np.mean(f_a) - np.mean(f_b))
print(round(difference, 4))

Plants on field B seem to be higher compared to field A. Are those differences due to chance?

### Resampling approach approach

In [ ]:
import pandas as pd

# concatenate together the two samples into a single one
a_b_together = pd.Series(np.concatenate([f_a, f_b]))


# initialize list for differences of means
dif_means = []
n_simulations = 50000
for i in range(n_simulations):
    # from that single sample, take 2 new random samples
    s_a = a_b_together.sample(len(f_a)) # random sample of the same size as field A
    s_b = a_b_together[~a_b_together.index.isin(s_a.index)] # the rest of the plants go to field B
    
    # get the difference of the means of these new 2 samples
    dif_means.append(s_a.mean()-s_b.mean())
    

# plot the distribution of the differences
dif_means = pd.Series(dif_means)
dif_means.hist(bins = 20)


# look at how extreme our original difference is (0.3)
count = 0

for d in dif_means:
    if d > difference:
        count += 1
    if d < -difference:
        count += 1
count

In [ ]:
# this can be interpreted as our p-value
p_val_resampl = count / n_simulations
print("p value resampling approach", round(p_val_resampl, 3))

### Formal approach: 2 sample t-test:

Here we calculate the t statistic, the degrees of freedom and the p value following khan's academy's video:

In [ ]:
# Calculate the t-statistic 

# the means of the two fields
mean_a = np.mean(f_a)
mean_b = np.mean(f_b)

# the variances of the two fields
var_a = np.var(f_a)
var_b = np.var(f_b)

# the sample sizes
n_a = len(f_a)
n_b = len(f_b)

# the standard error
# (remember: this is an estimate of the standard deviation of the "sampling distribution
#  of the differences of means" - the histogram we plotted above)
se = np.sqrt(var_a/n_a + var_b/n_b)

# the t statistic
t_statistic = (mean_a - mean_b) / se
print("t statistic:", t_statistic)

# calculate the p-value of the t statistic, 
# using the cumulative distribution function:
deg_free = np.min([n_a, n_b]) - 1
#deg_free = 23 # Scipy seems to use this value for degrees of freedom

print("degrees of freedom:", deg_free)

p_val = stats.t.cdf(t_statistic, deg_free)

# we're doing a two-tailed test, so we need to double the result to get our p-value
# (we need to account for 'equal or more extreme cases' in both sides of the distribution)
p_val = p_val*2
print("p value:", round(p_val, 4))

In [ ]:
alpha = 0.05

if(p_val < alpha):
    print("Null Hypothesis Rejected")
else:
    print("Failed to Reject the Null Hypothesis")

### Using scipy:



In [ ]:
t_stat, p_value = stats.ttest_ind(f_a, f_b)

print("P value for Scipy's t-test:", round(p_value, 3))

The p-value of Scipy's test with the default parameters matches closely the one from our resampling approach, but it is a bit different from the "formal approach" that Khan Academy shows on their video (`0.236`). 

This is because there are some slightly different variations to the traditional t-test called. In particular, the "Welch's t-test", which is a more "conservative approach" (outputs higher p-values) for samples with different variances and/or sample sizes (https://en.wikipedia.org/wiki/Welch%27s_t-test). This is the test that Khan Academy follows, and can be replicated with Scipy by setting the parameter `equal_var` to `False`. Now the p-value obtained is closer to the result in the video:

In [ ]:
t_statistic, p_value = stats.ttest_ind(f_a,
                                       f_b,
                                       equal_var=False)

print("P value for Welch's t-test:", round(p_value, 4))

If you are still wondering why the p-value is not _exactly the same_ as the one in the video, the answer lies in the degrees of freedom used to compute the t-distribution: Khan Academy uses again a "conservative approach" (the smaller sample size - 1) to avoid using a [slightly more complicated equation](https://en.wikipedia.org/wiki/Welch%E2%80%93Satterthwaite_equation) in an educational video. 

Unfortunately, Scipy does not clarify in its documentation how are the degrees of freedom computed, but it seems that for these samples they end up using a value close to `23`. If you go back to the code where we perform the test following Khan Academy's "formal approach" and you replace the degrees of freedom for `23`, you will see how the p-values finally match.

If you have followed the notebook to this point, you might be asking yourself: should I care about these different variations of the same statistical test, these slight differences in p-values, different ways to compute the degrees of freedom of a t-distribution, conservative vs. non-conservative approaches..? The answer is: if you want to be a fully rounded Data Analyst or Data Scientist in a digital company who has an eye on the business, who's fluent in at least a couple of programming languages, who's oriented towards driving actions in a fast-paced environment, and you're in the first steps of your learning path... then probably no. The opportunity cost of learning such details is just too high! The small differences we saw get completely irrelevant as sample sizes increase, and chances are you'll work with larger datasets and not just twenty plants.

There are still plenty of statistical important notions to learn when it comes to implementing A/B Testing. If you haven't already, watch [this Stanford Seminar](https://www.youtube.com/watch?v=AJX4W3MwKzU&t=112s&ab_channel=stanfordonline) on how an A/B Testing Software company like [Optimizely](https://www.optimizely.com/) uses Statistics to help their users interpret the results of a test.